Step 0 : Paths, Modules and Functions

In [1]:
#%% Files

# Path to genome
genome = '/home/valentin-grenet/Bureau/Données/Resources_yann/GCF_009389715.1_palm_55x_up_171113_PBpolish2nd_filt_p_genomic.fna'
# Path to RepeatMasker hits of all hits (rare + common TEs)
Repeat_all_data = '/home/valentin-grenet/Bureau/Données/Resources_yann/Repeat_LTR_all.out'
# Path to RepeatMasker hits of cdhit (commons TEs, used to make the consensus sequences)
Repeat_cdhit_data = '/home/valentin-grenet/Bureau/Données/Resources_yann/Repeat_LTR_cdhit.out'
# Path to Inpactor hits of all hits (rare + common TEs)
initial_library = '/home/valentin-grenet/Bureau/Données/Resources_yann/initial_library.fasta'
# Path to Inpactor hits of cdhit (commons TEs, used to make the consensus sequences)
final_library = '/home/valentin-grenet/Bureau/Données/Resources_yann/Final_library.fasta'
# Path to sequences of genome contigs
contig_dir = '/home/valentin-grenet/Bureau/Données/genome_sequences'
# Path to sequences of consensus
sequences_dir = '/home/valentin-grenet/Bureau/Données/LTR_cdhit_sequences'
# Path to libraries
libraries_dir = '/home/valentin-grenet/Bureau/Données/Gene_libraries/'
# Path to Dante classification
Dante_data = '/home/valentin-grenet/Bureau/Données/Resources_yann/DANTE_initial_like.txt'
# Path to output directory
results_dir = '/home/valentin-grenet/Bureau/Données/cdhit_classification'
# Path to MrBayes
mb = "/home/valentin-grenet/Bureau/Outils/MrBayes/src/mb"


#%% Modules

import os
import subprocess

from Bio import SeqIO                           # Used to have a uniform interface for input and output sequence file formats
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord             # SeqRecord will be the object of the sequence file during the analysis in code


### Step 1 : Extraction of RepeatMasker hit coordinates

In [ ]:
def ExtractCoordinatesRepeat(file, dico_hits):     # file = masking_hits
    '''Function used to extract all hit coordinates obtained with RepeatMasker. Those hits are stored followingly :
    dico_repeat_hits = {"consensus1" : [{hit1}, {hit2}, {hit3}, ...], "consensus2" : [{hit1}, {hit2}, {hit3}, ...], ...}
    hit represents a dictionnary storing informations about a RepeatMasker hit as the following example :
    {'contig': 'NC_016740.1', 'contig_start': '102549', 'contig_stop': '103252', 'strand': 'C', 'TE_start': '5985', 'TE_stop': '5278', 'perc_id': 62.0, 'index': 3}'''

    index=0
    for line in open(file, "r"):
        index+=1                # line indices of the hit in the RepeatMasker hits file (begin with 1)
        hit = {}
        if line[0]=="#":    # skip header lines
            continue

        column = []
        elements = line.split(" ")
        for i in range(0,len(elements)):
            if elements[i] != "":
                column.append(elements[i])      # adding each column of the tsv file in list
        if column[9] not in dico_hits:      # add the consensus to dico_repeat_hits
            dico_hits[column[9]] = []
            # print(column[9])
        hit["contig"] = column[4]
        hit["contig_start"] = column[5]
        hit["contig_stop"] = column[6]
        hit["strand"] = column[8]
        strand = 0
        if hit["strand"] == "C":            # used to manage TE_start and TE_stop (reversed with reverse strand)
            strand = 1
        hit["TE_start"] = column[11+strand]
        hit["TE_stop"] = column[12+strand]
        hit["perc_id"] = 100 - sum(map(float, column[1:3]))
        hit["index"] = index
        dico_hits[column[9]].append(hit)        # append the hit to repeat_dico_hits for the right consensus
    return(dico_hits)

dico_repeat_hits = {}
# dico_repeat_hits = ExtractCoordinatesRepeat(Repeat_all_data, dico_repeat_hits)
dico_repeat_hits = ExtractCoordinatesRepeat(Repeat_cdhit_data, dico_repeat_hits)

In [ ]:
# if you want to check the dictionnary, change the consensus variable with a proper name

consensus = "consensus_Cluster_67_subfam_2"
for hit in dico_repeat_hits[consensus]:
    print(hit)

### Step 2 : Extraction of sequences (LTRs, genome and consensus)

In [ ]:
def ExtractLengths(consensus, file, dico_sequences):
    '''Extract the length of a sequence from a fasta file to a dictionnary of consensus names as keys.
    The consensus name as to be provided with an existing dictionnary'''
    dico_sequences[consensus] = 0
    if file in os.listdir('.'):
        for line in open(file, "r"):
            if line[0] != ">":
                dico_sequences[consensus] += len(line)-1
    return dico_sequences

# dico_genome = ExtractSequences(genome, {})
# dico_genome = {seq_record.id: seq_record for seq_record in SeqIO.parse(genome, "fasta")}          # dictionnary of contig sequences, not used here
# dico_TEs = ExtractSequences(initial_library, {})

os.chdir(sequences_dir)
dico_consensus = {}; dico_5LTR = {}; dico_RT = {}; dico_RH = {}; dico_PROT = {}; dico_INT = {}; dico_GAG = {}; dico_3LTR = {}
for consensus in os.listdir('.'):           # need a directory with only consensus directories
    if "c" in consensus:
        os.chdir(consensus)
        if "c" in consensus:
            dico_consensus = ExtractLengths(consensus, consensus + ".fasta", dico_consensus)
            dico_5LTR = ExtractLengths(consensus, consensus + ".5-LTR.fasta", dico_5LTR)
            dico_RT = ExtractLengths(consensus, consensus + ".RT.fasta", dico_RT)
            dico_RH = ExtractLengths(consensus, consensus + ".RH.fasta", dico_RH)
            dico_PROT = ExtractLengths(consensus, consensus + ".PROT.fasta", dico_PROT)
            dico_INT = ExtractLengths(consensus, consensus + ".RT.fasta", dico_INT)
            dico_GAG = ExtractLengths(consensus, consensus + ".GAG.fasta", dico_GAG)
            dico_3LTR = ExtractLengths(consensus, consensus + ".3-LTR.fasta", dico_3LTR)
        os.chdir("..")

dico_length = {}        # dico_length = for each consensus as key, a dictionnary with the length of all elements (genes, total, LTRs)
for consensus in dico_consensus:
    dico_length[consensus] = {"complete":dico_consensus[consensus], 
                              "5-LTR":dico_5LTR[consensus],
                              "RT":dico_RT[consensus],
                              "RH":dico_RH[consensus],
                              "PROT":dico_PROT[consensus],
                              "INT":dico_INT[consensus],
                              "GAG":dico_GAG[consensus],
                              "3-LTR":dico_3LTR[consensus],}

In [ ]:
# if you want to check your dictionnary

for consensus in dico_consensus:
    print(consensus, dico_length[consensus])

### Step 3 : Classification of LTRs

#### Step 3.1 : Simple recognition - Classification with length of LTR and full TE
This first version only requires the length of elements to classify them.
The initial percentage of coverage accepted is 80 % (0.8 in CheckLength)

In [ ]:
def CheckLength(hit, length, count):
    '''From the length of the TE hit provided and the expected lengths of LTRs and complete
    consensus sequence, the function proceeds to the classifcation of the hit.
    - trunc : hit too short compared to LTR length
    - paired : hit long enough to match complete sequence length, so considered with both LTRs
    - single : hit too short to have both LTRs, but a great amount of internal sequence
    - solo : hit with one LTR and some internal sequence'''
    length_hit = int(hit["stop"]) - int(hit["start"])
    if length_hit < 0.8*length["5-LTR"]:
        count[0] += 1
        return "trunc", count
    elif length_hit > 0.8*length["complete"]:
        count[3] += 1
        return "paired", count
    elif length_hit > 0.8*(length["complete"]-length["5-LTR"]):
        count[2] += 1
        return "single", count
    else:
        count[1] += 1
        return "solo", count

os.chdir(sequences_dir)
headers = ["index", "contig", "start", "stop", "length", "strand", "element"]        # informations given for each hit in output files
dico_count = {}

for consensus in dico_repeat_hits :
    print(consensus)
    os.chdir(consensus)
    output_consensus = open(consensus + ".LTR_types.tsv", "w")
    output_consensus.write("\t".join(headers) + "\n")
    count = [0,0,0,0]             # count the number of paired, single, solo and truncated elements
    
    for hit in dico_repeat_hits[consensus]:
        status, count = CheckLength(hit, dico_length[consensus], count)         # status = LTR classification
        line = [str(hit["index"]), hit["contig"], hit["start"], hit["stop"], str(int(hit["stop"])-int(hit["start"])), hit["strand"], status]
        output_consensus.write("\t".join(line)+"\n")
    
    output_consensus.close()
    dico_count[consensus] = count           # one count list for each consensus
    os.chdir("..")

output_count = open("annotation_counts.tsv", "w")
headers = ["consensus", "length", "trunc", "solo", "single", "paired"]
output_count.write("\t".join(headers) + "\n")

for consensus in dico_count:
    output_count.write(consensus + "\t" + str(dico_length[consensus]["complete"]) + "\t" + "\t".join(str(i) for i in dico_count[consensus]) + "\n")   

#### Step 3.2 : Advanced recognition - Classification with length and coordinates
This second version is still using the length of elements to classify each hits, but also use coordinates of hits to identify the presence or not of LTRs, providing a more efficient classification, especially for paired ones. However, some problems remain (as taking the strand sense into account) and the next version is the more efficient.


In [ ]:
def ClassLTR(hit, consensus_length, consensus_hits, count):
    '''From the length of the TE hit provided and the expected lengths of LTRs and complete
    consensus sequence, the function proceeds to the classifcation of the hit.
    The coordinates of hit with the TE are also taken into account to precise the classification.
    - complete (paired in 3.1) : hit long enough to match complete sequence length, so considered with both LTRs
    if not complete, presence of LTRs is checked.
    - paired : two complete LTRs have been identified with 2 different hits
    - single : one LTR identified, and also some internal sequence
    - solo : one LTR identified but not much internal sequence with it
    - trunc : hit too short compared to LTR length'''
    if CheckComplete(hit, consensus_length):
        count[4] += 1
        return [hit], "complete", count
    else:
        if CheckLTR(hit, consensus_length):
            if CheckPaired(hit, consensus_length, consensus_hits):
                TE_hits = BuildTE(hit, consensus_length, consensus_hits)
                count[3] += 1
                return TE_hits, "paired", count
            else:
                if CheckSingle(hit, consensus_length, consensus_hits):
                    TE_hits = BuildTE(hit, consensus_length, consensus_hits)
                    count[2] += 1
                    return TE_hits, "single", count
                else:
                    count[1] += 1
                    return [hit], "solo", count
        else:
            count[0] += 1
            return [], "trunc", count

def CheckComplete(hit, consensus_length):
    '''Check the length of the hit and compare hit to the consensus complete sequence length.
    Coverage percentage of 80 % minimum.'''
    length_hit = int(hit["contig_stop"]) - int(hit["contig_start"])
    return length_hit >= 0.8*consensus_length["complete"]

def CheckLTR(hit, consensus_length):
    '''Based on TE hit coordinates (with what matches the hit on the TE),
    we estimate the coverage of a single LTR based on its length. If the coverage is sufficient
    (75 % of identity on 50 % of the LTR), we consider that the LTR as been covered'''
    if hit["strand"] == "+" :
        # testing if the coordinates are matching with an LTR covering and not an internal sequence covering
        if int(hit["TE_start"]) <= consensus_length["5-LTR"] \
        or int(hit["TE_stop"]) >= consensus_length["complete"]-consensus_length["3-LTR"]:
            # testing if the identity and coverage are sufficient
            return int(hit["TE_stop"])-int(hit["TE_start"]) >= 0.5*consensus_length["5-LTR"] \
                and int(hit["perc_id"]) >= 75
        else:
            return False
    else:
        # testing if the coordinates are matching with an LTR covering and not an internal sequence covering
        if int(hit["TE_stop"]) <= consensus_length["5-LTR"] \
        or int(hit["TE_start"]) >= consensus_length["complete"]-consensus_length["3-LTR"]:
            # testing if the identity and coverage are sufficient
            return int(hit["TE_start"])-int(hit["TE_stop"]) >= 0.5*consensus_length["5-LTR"] \
                and int(hit["perc_id"]) >= 75
        else:
            return False

def CheckPaired(query_hit, consensus_length, consensus_hits):
    '''From the first LTR identified, and according to its strand sense and its classification (5' or 3'),
    we are searching the second LTR.
    Not working because of errors with the strand sense.'''
    if int(query_hit["TE_start"]) < consensus_length["5-LTR"] : 
        if query_hit["strand"] == "+" :
            coord_start = int(query_hit["contig_start"]) - int(query_hit["TE_start"])
        else:
            coord_start = int(query_hit["contig_start"]) + int(query_hit["TE_start"]) - consensus_length["complete"]
        # coord_min = coord_start + consensus_length["complete"] - (1.2*consensus_length["3-LTR"])
        coord_max = coord_start + consensus_length["complete"] + (0.2*consensus_length["3-LTR"])
        print(query_hit["index"], coord_start, coord_max)
        for other_hit in consensus_hits:
            if coord_start <= int(other_hit["contig_stop"]) <= coord_max \
            and query_hit["contig"] == other_hit["contig"] \
            and query_hit["index"] != other_hit["index"]:               # added for smaller TE paired anyway (linked with suppression of coord_min)
                if CheckLTR(other_hit, consensus_length):
                    return True
    else:                                                                   # else we have the 3-LTR
        if query_hit["strand"] == "+" :
            coord_start = int(query_hit["contig_stop"]) - int(query_hit["TE_stop"]) + consensus_length["complete"]
        else:
            coord_start = int(query_hit["contig_stop"]) + int(query_hit["TE_stop"])
        coord_min = coord_start - consensus_length["complete"] - (0.2*consensus_length["3-LTR"])
        # coord_max = coord_start - consensus_length["complete"] + (1.2*consensus_length["3-LTR"])
        print(query_hit["index"], coord_start, coord_min)
        for other_hit in consensus_hits:
            if coord_min <= int(other_hit["contig_start"]) <= coord_start \
            and query_hit["contig"] == other_hit["contig"] \
            and query_hit["index"] != other_hit["index"]:               # added for smaller TE paired anyway (linked with suppression of coord_max)
                if CheckLTR(other_hit, consensus_length):
                    return True
    return False

def CheckSingle(query_hit, consensus_length, consensus_hits):
    length_hit = int(hit["contig_stop"]) - int(hit["contig_start"])
    if length_hit >= consensus_length["5-LTR"]+500:
        return True
    else:
        if int(query_hit["TE_start"]) < consensus_length["5-LTR"] :          # change due to error of pairing with itself
            if query_hit["strand"] == "+" :
                coord_start = int(query_hit["contig_start"]) - int(query_hit["TE_start"]) + consensus_length["5-LTR"]
            else:
                coord_start = int(query_hit["contig_start"]) + int(query_hit["TE_start"]) - consensus_length["complete"] + consensus_length["5-LTR"]
            for other_hit in consensus_hits:
                if coord_start <= int(other_hit["contig_start"]) <= coord_start+500 \
                and query_hit["contig"] == other_hit["contig"]:
                    return True
        else:
            if query_hit["strand"] == "+" :
                coord_start = int(query_hit["contig_stop"]) - int(query_hit["TE_stop"]) + consensus_length["complete"] - consensus_length["5-LTR"]
            else:
                coord_start = int(query_hit["contig_stop"]) + int(query_hit["TE_stop"]) - consensus_length["5-LTR"]
            for other_hit in consensus_hits:
                if coord_start-500 <= int(other_hit["contig_stop"]) <= coord_start \
                and query_hit["contig"] == other_hit["contig"]:
                    return True
        return False

def BuildTE(query_hit, consensus_length, consensus_hits):
    TE_hits = []
    if int(query_hit["TE_start"]) < consensus_length["5-LTR"] :          # change due to error of pairing with itself
        if query_hit["strand"] == "+" :
            coord_start = int(query_hit["contig_start"]) - int(query_hit["TE_start"])
        else:
            coord_start = int(query_hit["contig_start"]) + int(query_hit["TE_start"]) - consensus_length["complete"]
        for other_hit in consensus_hits:
            # print(other_hit["contig_start"])
            if coord_start <= int(other_hit["contig_start"]) <= coord_start+1.2*consensus_length["complete"] \
            and query_hit["contig"] == other_hit["contig"]:
                TE_hits.append(other_hit)
    else:
        if query_hit["strand"] == "+" :
            coord_start = int(query_hit["contig_stop"]) - int(query_hit["TE_stop"]) + consensus_length["complete"]
        else:
            coord_start = int(query_hit["contig_stop"]) + int(query_hit["TE_stop"])
        for other_hit in consensus_hits:
            if coord_start-1.2*consensus_length["complete"] <= int(other_hit["contig_stop"]) <= coord_start \
            and query_hit["contig"] == other_hit["contig"]:
                TE_hits.append(other_hit)
    return TE_hits



os.chdir(sequences_dir)
headers = ["index", "contig", "start", "stop", "length", "strand", "id_percentage", "TE_start", "TE_stop", "classification", "index", "start", "stop", "length", "strand", "id_percentage", "TE_start", "TE_stop"]
dico_count = {}

for consensus in dico_repeat_hits :
    print(consensus)
    os.chdir(consensus)
    output_consensus = open(consensus + ".LTR_classif.tsv", "w")
    output_consensus.write("\t".join(headers) + "\n")
    count = [0,0,0,0,0]
    hits_used = []
    
    for hit in dico_repeat_hits[consensus]:
        if hit in hits_used:
            continue
        else:
            TE_hits, status, count = ClassLTR(hit, dico_length[consensus], dico_repeat_hits[consensus], count)
            line = [str(hit["index"]), hit["contig"], hit["contig_start"], hit["contig_stop"], str(int(hit["contig_stop"])-int(hit["contig_start"])), hit["strand"], str(hit["perc_id"]), hit["TE_start"], hit["TE_stop"], status]
            output_consensus.write("\t".join(line))
            if len(TE_hits) == 0 :
                output_consensus.write('\tno match\n')
            else:
                new_line = '\t'
                for other_hit in TE_hits:
                    hits_used.append(other_hit)
                    line = [str(other_hit["index"]), other_hit["contig_start"], other_hit["contig_stop"], str(int(other_hit["contig_stop"])-int(other_hit["contig_start"])), other_hit["strand"], str(other_hit["perc_id"]), other_hit["TE_start"], other_hit["TE_stop"]]
                    output_consensus.write(new_line + "\t".join(line) + "\n")
                    new_line = 10*'\t'

    
    output_consensus.close()
    dico_count[consensus] = count
    os.chdir("..")

output_count = open("annotation_counts.tsv", "w")
headers = ["consensus", "length", "trunc", "solo", "single", "paired", "complete"]
output_count.write("\t".join(headers) + "\n")

for consensus in dico_count:
    output_count.write(consensus + "\t" + str(dico_length[consensus]["complete"]) + "\t" + "\t".join(str(i) for i in dico_count[consensus]) + "\n")   

#### Step 3.3 : Major updates on recognition

The previous script had interessant results, so we kept the same method and brought ameliorations :
- for reverse strand sense, uncomplete fragment then added for a single or a paired LTR are still written as trunc -> option to keep them in memory, and then write them as trunc or in the paired or single LTR, and modify the count list
- add the option to recheck the completion of the LTR analyzed to pass it from paired to complete (or fragmented)
- manage the cases where the LTR is complete, but too small to be considered as a complete (internal sequences truncated) --> considered as shrunked
- change of management between LTR scan on the left or on the right, because it was not working well before

comment on the efficiency :
the script will never be perfect as the threshold will always miss some LTR which should be considered as a classification above (false negative) and consider some LTR in a classification above where they should be (false positive)

In [ ]:
def SearchSense(strand, TE_start, TE_stop, LTR_length):
    '''Function to define the sense of research of the second LTR'''
    sign = strand == "+"
    number = TE_start < LTR_length or TE_stop < LTR_length
    if sign and number:
        return True
    elif not sign and not number:
        return True
    else:
        return False
    
def FixCount(status, count):
    '''Function to uncount a smaller hit first counted but then taken by a reversed longer hit
    (example : a trunc hit is part of the internal sequence of a paired hit)'''
    list_status = ["trunc", "solo", "single", "paired", "fragmented", "shrunked", "complete"]
    count[list_status.index(status)] -= 1
    return count

def ClassLTR(hit, consensus_length, consensus_hits, count):
    '''From the length of the TE hit provided and the expected lengths of LTRs and complete
    consensus sequence, the function proceeds to the classifcation of the hit.
    The coordinates of hit with the TE are also taken into account to precise the classification.
    - complete : hit long enough to match complete sequence length, so considered with both LTRs
    if not complete, presence of LTRs is checked.
    - shrunked : hit not long enough to be considered as complete, but the single hit as both LTRs
    according to its TE hit coordinates (case of internal sequence shortened)
    - fragmented : two complete LTRs with 2 different hits, and internal sequence is 
    sufficiently covered as well (so a complete TE on several hits)
    - paired : two complete LTRs have been identified with 2 different hits, but internal sequence is lacking
    - single : one LTR identified, and also some internal sequence
    - solo : one LTR identified but not much internal sequence with it
    - trunc : hit too short compared to LTR length'''
    if CheckComplete(hit, consensus_length):
        count[6] += 1
        return [hit], "complete", count
    elif CheckShrunked(hit, consensus_length):            # peut-être clippé
        count[5] += 1
        return [hit], "shrunked", count
    elif CheckLTR(hit, consensus_length):
        if CheckPaired(hit, consensus_length, consensus_hits):
            TE_hits = BuildTE(hit, consensus_length, consensus_hits)
            if CheckFragmented(TE_hits, consensus_length):
                count[4] += 1
                return TE_hits, "fragmented", count
            else:
                count[3] += 1
                return TE_hits, "paired", count
        elif CheckSingle(hit, consensus_length, consensus_hits):
            TE_hits = BuildTE(hit, consensus_length, consensus_hits)
            count[2] += 1
            return TE_hits, "single", count
        else:
            count[1] += 1
            return [hit], "solo", count
    else:
        count[0] += 1
        return [], "trunc", count

def CheckComplete(hit, consensus_length):
    '''Check the length of the hit and compare hit to the consensus complete sequence length.
    Coverage percentage of 80 % minimum.'''
    length_hit = int(hit["contig_stop"]) - int(hit["contig_start"])
    return length_hit >= 0.8*consensus_length["complete"]

def CheckShrunked(hit, consensus_length):
    '''After being checked as not complete (not long enough), we verify if the hit has both LTRs anyway
    (case of truncated internal sequence, but both LTRs are present)'''
    coord_5LTR = 0.5*consensus_length["5-LTR"]
    coord_3LTR = consensus_length["complete"] - 0.5*consensus_length["3-LTR"]
    if hit["strand"] == "+":
        return int(hit["TE_start"]) < coord_5LTR and int(hit["TE_stop"]) > coord_3LTR
    else:
        return int(hit["TE_stop"]) < coord_5LTR and int(hit["TE_start"]) > coord_3LTR

def CheckLTR(hit, consensus_length):
    '''Based on TE hit coordinates (with what matches the hit on the TE),
    we estimate the coverage of a single LTR based on its length. If the coverage is sufficient
    (75 % of identity on 50 % of the LTR), we consider that the LTR as been covered'''
    if hit["strand"] == "+" :
        # testing if the coordinates are matching with an LTR covering and not an internal sequence covering
        if int(hit["TE_start"]) <= consensus_length["5-LTR"] \
        or int(hit["TE_stop"]) >= consensus_length["complete"]-consensus_length["3-LTR"]:
            # testing if the identity and coverage are sufficient
            return int(hit["TE_stop"])-int(hit["TE_start"]) >= 0.5*consensus_length["5-LTR"] \
                and int(hit["perc_id"]) >= 75
        else:
            return False
    else:
        # testing if the coordinates are matching with an LTR covering and not an internal sequence covering
        if int(hit["TE_stop"]) <= consensus_length["5-LTR"] \
        or int(hit["TE_start"]) >= consensus_length["complete"]-consensus_length["3-LTR"]:
            # testing if the identity and coverage are sufficient
            return int(hit["TE_start"])-int(hit["TE_stop"]) >= 0.5*consensus_length["5-LTR"] \
                and int(hit["perc_id"]) >= 75
        else:
            return False

def CheckPaired(query_hit, consensus_length, consensus_hits):
    '''From the first LTR identified with the hit coordinates, we build an interval of coordinates
    on the genome in which we have to find the second LTR (so use of CheckLTR for any hit found in this interval)'''
    # case of searching the second LTR on the right (3'LTR for a 5'LTR on +, or 5'LTR for a 3'LTR on -)
    if SearchSense(query_hit["strand"], int(query_hit["TE_start"]), int(query_hit["TE_stop"]), consensus_length["5-LTR"]):
        # coord_start = estimated first base of 5'LTR or last base of 3'LTR from TE and contig coordinates
        if query_hit["strand"] == "+" :
            coord_start = int(query_hit["contig_start"]) - int(query_hit["TE_start"])
        else:
            coord_start = int(query_hit["contig_start"]) + int(query_hit["TE_start"]) - consensus_length["complete"]
        coord_min = coord_start + consensus_length["complete"] - (1.2*consensus_length["3-LTR"])
        coord_max = coord_start + consensus_length["complete"] + (1.2*consensus_length["3-LTR"])
        for other_hit in consensus_hits:
            if coord_min <= int(other_hit["contig_stop"]) <= coord_max \
            and query_hit["contig"] == other_hit["contig"] \
            and query_hit["index"] != other_hit["index"]:               # avoid pairing with itself
                if CheckLTR(other_hit, consensus_length):
                    return True
    # case of searching the second LTR on the left (5'LTR for a 3'LTR on +, or 3'LTR for a 5'LTR on -)
    else:
        # coord_start = estimated first base of 5'LTR or last base of 3'LTR from TE and contig coordinates
        if query_hit["strand"] == "+" :
            coord_start = int(query_hit["contig_stop"]) - int(query_hit["TE_stop"]) + consensus_length["complete"]
        else:
            coord_start = int(query_hit["contig_stop"]) + int(query_hit["TE_stop"])
        coord_min = coord_start - consensus_length["complete"] - (1.2*consensus_length["3-LTR"])
        coord_max = coord_start - consensus_length["complete"] + (1.2*consensus_length["3-LTR"])
        for other_hit in consensus_hits:
            if coord_min <= int(other_hit["contig_start"]) <= coord_max \
            and query_hit["contig"] == other_hit["contig"] \
            and query_hit["index"] != other_hit["index"]:               # avoid pairing with itself
                if CheckLTR(other_hit, consensus_length):
                    return True
    return False

def CheckFragmented(TE_hits, consensus_length):
    '''Check if the collection of hits building one TE is covering a great amount of internal sequence
    (coverage > 80 % as a complete TE)'''
    length_covered = 0
    for hit in TE_hits:
        if length_covered == 0:
            length_covered = int(hit["contig_stop"]) - int(hit["contig_start"])
            coord_max = int(hit["contig_stop"])
        else:
            length_covered += int(hit["contig_stop"]) - max(coord_max, int(hit["contig_start"]))
    return length_covered >= 0.8*consensus_length["complete"]

def CheckSingle(query_hit, consensus_length, consensus_hits):
    '''Check if the hit (or a near small hit) is covering some internal sequence in addition to its LTR'''
    length_hit = int(hit["contig_stop"]) - int(hit["contig_start"])
    if length_hit >= consensus_length["5-LTR"]+500:
        return True
    else:
        # case of searching a second hit on the right (5'LTR on +, or 3'LTR on -)
        if SearchSense(query_hit["strand"], int(query_hit["TE_start"]), int(query_hit["TE_stop"]), consensus_length["5-LTR"]):
            # coord_start = estimated last base of 5'LTR or first base of 3'LTR from TE and contig coordinates
            if query_hit["strand"] == "+" :
                coord_start = int(query_hit["contig_start"]) - int(query_hit["TE_start"]) + consensus_length["5-LTR"]
            else:
                coord_start = int(query_hit["contig_start"]) + int(query_hit["TE_start"]) - consensus_length["complete"] + consensus_length["5-LTR"]
            for other_hit in consensus_hits:
                if coord_start <= int(other_hit["contig_start"]) <= coord_start+500 \
                and query_hit["contig"] == other_hit["contig"]:
                    return True
        # case of searching a second hit on the left (3'LTR on +, or 5'LTR on -)
        else:
            # coord_start = estimated last base of 5'LTR or first base of 3'LTR from TE and contig coordinates
            if query_hit["strand"] == "+" :
                coord_start = int(query_hit["contig_stop"]) - int(query_hit["TE_stop"]) + consensus_length["complete"] - consensus_length["5-LTR"]
            else:
                coord_start = int(query_hit["contig_stop"]) + int(query_hit["TE_stop"]) - consensus_length["5-LTR"]
            for other_hit in consensus_hits:
                if coord_start-500 <= int(other_hit["contig_stop"]) <= coord_start \
                and query_hit["contig"] == other_hit["contig"]:
                    return True
        return False

def BuildTE(query_hit, consensus_length, consensus_hits):
    '''From the first hit identified with hit coordinates, we build an interval of coordinates
    on the genome (corresponding to the complete TE length), in which we find other hits of the
    same consensus. The query and found hits are considered building the same TE.'''
    TE_hits = []            # list of hits added with the query hit
    # case of searching a hits on the right (5'LTR on +, or 3'LTR on -)
    if SearchSense(query_hit["strand"], int(query_hit["TE_start"]), int(query_hit["TE_stop"]), consensus_length["5-LTR"]):
        # coord_start = estimated first base of 5'LTR or last base of 3'LTR from TE and contig coordinates
        if query_hit["strand"] == "+" :
            coord_start = int(query_hit["contig_start"]) - int(query_hit["TE_start"])
        else:
            coord_start = int(query_hit["contig_start"]) + int(query_hit["TE_start"]) - consensus_length["complete"]
        for other_hit in consensus_hits:
            # print(other_hit["contig_start"])
            if coord_start <= int(other_hit["contig_start"]) <= coord_start+1.2*consensus_length["complete"] \
            and query_hit["contig"] == other_hit["contig"]:
                TE_hits.append(other_hit)
    # case of searching a hits on the left (3'LTR on +, or 5'LTR on -)
    else:
        # coord_start = estimated first base of 5'LTR or last base of 3'LTR from TE and contig coordinates
        if query_hit["strand"] == "+" :
            coord_start = int(query_hit["contig_stop"]) - int(query_hit["TE_stop"]) + consensus_length["complete"]
        else:
            coord_start = int(query_hit["contig_stop"]) + int(query_hit["TE_stop"])
        for other_hit in consensus_hits:
            if coord_start-1.2*consensus_length["complete"] <= int(other_hit["contig_stop"]) <= coord_start \
            and query_hit["contig"] == other_hit["contig"]:
                TE_hits.append(other_hit)
    return TE_hits



os.chdir(sequences_dir)
# headers for the LTR classification files for each consensus
headers = ["index", "contig", "start", "stop", "length", "strand", "id_percentage", "TE_start", "TE_stop", "classification", "index", "start", "stop", "length", "strand", "id_percentage", "TE_start", "TE_stop"]
dico_count = {}

for consensus in dico_repeat_hits :
    print(consensus)
    os.chdir(consensus)
    output_consensus = open(consensus + ".LTR_classif.tsv", "w")
    output_consensus.write("\t".join(headers) + "\n")
    count = [0,0,0,0,0,0,0]             # count the number of each LTR class, one list per consensus
    hits_used = []                      # used to avoid repetitions of hits written in the output files
    list_lines = []                     # used to write the output file for each consensus (the list contains all lines for one consensus)
    
    for hit in dico_repeat_hits[consensus]:
        # manage repetitions in + sense (a hit building another TE is in hits_used)
        if hit in hits_used:
            continue
        else:
            TE_hits, status, count = ClassLTR(hit, dico_length[consensus], dico_repeat_hits[consensus], count)
            ref_line = [str(hit["index"]), hit["contig"], hit["contig_start"], hit["contig_stop"], str(int(hit["contig_stop"])-int(hit["contig_start"])), hit["strand"], str(hit["perc_id"]), hit["TE_start"], hit["TE_stop"], status]
            list_lines.append(ref_line)
            # in case of a trunc LTR
            if len(TE_hits) == 0 :
                list_lines.append(["\tno_match\n"])
            else:
                new_line = ''
                for other_hit in TE_hits:
                    hits_used.append(other_hit)
                    # manage repetitions in - sense (smaller hits taking part in another TE and already written have to be deleted)
                    for line in list_lines:
                        if str(other_hit["index"]) == line[0] and line != ref_line:         # ref_line already in the list, but is not a repetition
                            del list_lines[list_lines.index(line):list_lines.index(line)+2]
                            count = FixCount(line[-1], count)
                    line = [new_line, str(other_hit["index"]), other_hit["contig_start"], other_hit["contig_stop"], str(int(other_hit["contig_stop"])-int(other_hit["contig_start"])), other_hit["strand"], str(other_hit["perc_id"]), other_hit["TE_start"], other_hit["TE_stop"], "\n"]
                    list_lines.append(line)
                    new_line = 9*'\t'

    for line in list_lines:
        output_consensus.write("\t".join(line))
    output_consensus.close()
    dico_count[consensus] = count
    os.chdir("..")

output_count = open("annotation_counts.tsv", "w")
headers = ["consensus", "length", "trunc", "solo", "single", "paired", "fragmented", "shrunked", "complete"]
output_count.write("\t".join(headers) + "\n")

for consensus in dico_count:
    output_count.write(consensus + "\t" + str(dico_length[consensus]["complete"]) + "\t" + "\t".join(str(i) for i in dico_count[consensus]) + "\n")   

#### Step 3.4 : Minor updates on recognition - 3.3 is working very well, so now we are making optimisations

Here is the same script as previously, but it is free for some modifications to bring the ameliorations below.

to do :
- uncomplete fragments added after the end of the LTR --> count them as trunc, or keep them as they could be part of the TE ? (ex : 34971 for c67_s2)
- sometimes, a complete fragment is considered as paired --> identify them with maybe a second check of complete, or the impossibility to add a too long fragment (ex : 154500 in consensus67_s2)
- consider the paired LTR too close to be considered as paired (important lack of internal sequence) (ex : TE of 9000 instead of 12000 for 5346 in c67_s2), but manage the threshold to not paired the LTR with himself if the LTR is too long
- parameters modified by the user

done :
- completion factor from 0.8 to 0.95 because for the next step, some complete fragments didn't have both LTR

comment on the efficiency :
the script will never be perfect as the threshold will always miss some LTR which should be considered as a classification above (false negative) and consider some LTR in a classification above where they should be (false positive)

In [ ]:
def SearchSense(strand, TE_start, TE_stop, LTR_length):
    sign = strand == "+"
    number = TE_start < LTR_length or TE_stop < LTR_length
    if sign and number:
        return True
    elif not sign and not number:
        return True
    else:
        return False
    
def FixCount(status, count):
    list_status = ["trunc", "solo", "single", "paired", "fragmented", "shrunked", "complete"]
    count[list_status.index(status)] -= 1
    return count

def ClassLTR(hit, consensus_length, consensus_hits, count):
    if CheckComplete(hit, consensus_length):
        count[6] += 1
        return [hit], "complete", count
    elif CheckShrunked(hit, consensus_length):            # peut-être clippé
        count[5] += 1
        return [hit], "shrunked", count
    elif CheckLTR(hit, consensus_length):
        if CheckPaired(hit, consensus_length, consensus_hits):
            TE_hits = BuildTE(hit, consensus_length, consensus_hits)
            if CheckFragmented(TE_hits, consensus_length):
                count[4] += 1
                return TE_hits, "fragmented", count
            else:
                count[3] += 1
                return TE_hits, "paired", count
        elif CheckSingle(hit, consensus_length, consensus_hits):
            TE_hits = BuildTE(hit, consensus_length, consensus_hits)
            count[2] += 1
            return TE_hits, "single", count
        else:
            count[1] += 1
            return [hit], "solo", count
    else:
        count[0] += 1
        return [], "trunc", count

def CheckComplete(hit, consensus_length):
    length_hit = int(hit["contig_stop"]) - int(hit["contig_start"])
    return length_hit >= 0.95*consensus_length["complete"]

def CheckShrunked(hit, consensus_length):
    coord_5LTR = 0.5*consensus_length["5-LTR"]
    coord_3LTR = consensus_length["complete"] - 0.5*consensus_length["3-LTR"]
    if hit["strand"] == "+":
        return int(hit["TE_start"]) < coord_5LTR and int(hit["TE_stop"]) > coord_3LTR
    else:
        return int(hit["TE_stop"]) < coord_5LTR and int(hit["TE_start"]) > coord_3LTR

def CheckLTR(hit, consensus_length):
    if hit["strand"] == "+" :
        if int(hit["TE_start"]) <= consensus_length["5-LTR"] \
        or int(hit["TE_stop"]) >= consensus_length["complete"]-consensus_length["3-LTR"]:
            return int(hit["TE_stop"])-int(hit["TE_start"]) >= 0.5*consensus_length["5-LTR"] \
                and int(hit["perc_id"]) >= 75
        else:
            return False
    else:
        if int(hit["TE_stop"]) <= consensus_length["5-LTR"] \
        or int(hit["TE_start"]) >= consensus_length["complete"]-consensus_length["3-LTR"]:
            return int(hit["TE_start"])-int(hit["TE_stop"]) >= 0.5*consensus_length["5-LTR"] \
                and int(hit["perc_id"]) >= 75
        else:
            return False

def CheckPaired(query_hit, consensus_length, consensus_hits):
    if SearchSense(query_hit["strand"], int(query_hit["TE_start"]), int(query_hit["TE_stop"]), consensus_length["5-LTR"]):          # change due to error of pairing with itself, so if we have the 5-LTR
        if query_hit["strand"] == "+" :
            coord_start = int(query_hit["contig_start"]) - int(query_hit["TE_start"])
        else:
            coord_start = int(query_hit["contig_start"]) + int(query_hit["TE_start"]) - consensus_length["complete"]
        coord_min = coord_start + consensus_length["complete"] - (1.2*consensus_length["3-LTR"])
        coord_max = coord_start + consensus_length["complete"] + (1.2*consensus_length["3-LTR"])
        if query_hit["contig"] == "NW_024067682.1":
            print(query_hit["index"], coord_min, coord_max)
        for other_hit in consensus_hits:
            if coord_min <= int(other_hit["contig_stop"]) <= coord_max \
            and query_hit["contig"] == other_hit["contig"] \
            and query_hit["index"] != other_hit["index"]:               # added for smaller TE paired anyway (linked with suppression of coord_min)
                if CheckLTR(other_hit, consensus_length):
                    return True
    else:                                                                   # else we have the 3-LTR
        if query_hit["strand"] == "+" :
            coord_start = int(query_hit["contig_stop"]) - int(query_hit["TE_stop"]) + consensus_length["complete"]
        else:
            coord_start = int(query_hit["contig_stop"]) + int(query_hit["TE_stop"])
        coord_min = coord_start - consensus_length["complete"] - (1.2*consensus_length["3-LTR"])
        coord_max = coord_start - consensus_length["complete"] + (1.2*consensus_length["3-LTR"])
        for other_hit in consensus_hits:
            if coord_min <= int(other_hit["contig_start"]) <= coord_max \
            and query_hit["contig"] == other_hit["contig"] \
            and query_hit["index"] != other_hit["index"]:               # added for smaller TE paired anyway (linked with suppression of coord_max)
                if CheckLTR(other_hit, consensus_length):
                    return True
    return False

def CheckFragmented(TE_hits, consensus_length):
    length_covered = 0
    for hit in TE_hits:
        if length_covered == 0:
            length_covered = int(hit["contig_stop"]) - int(hit["contig_start"])
            coord_max = int(hit["contig_stop"])
        else:
            length_covered += int(hit["contig_stop"]) - max(coord_max, int(hit["contig_start"]))
    return length_covered >= 0.8*consensus_length["complete"]

def CheckSingle(query_hit, consensus_length, consensus_hits):
    length_hit = int(hit["contig_stop"]) - int(hit["contig_start"])
    if length_hit >= consensus_length["5-LTR"]+500:
        return True
    else:
        if SearchSense(query_hit["strand"], int(query_hit["TE_start"]), int(query_hit["TE_stop"]), consensus_length["5-LTR"]):           # change due to error of pairing with itself
            if query_hit["strand"] == "+" :
                coord_start = int(query_hit["contig_start"]) - int(query_hit["TE_start"]) + consensus_length["5-LTR"]
            else:
                coord_start = int(query_hit["contig_start"]) + int(query_hit["TE_start"]) - consensus_length["complete"] + consensus_length["5-LTR"]
            for other_hit in consensus_hits:
                if coord_start <= int(other_hit["contig_start"]) <= coord_start+500 \
                and query_hit["contig"] == other_hit["contig"]:
                    return True
        else:
            if query_hit["strand"] == "+" :
                coord_start = int(query_hit["contig_stop"]) - int(query_hit["TE_stop"]) + consensus_length["complete"] - consensus_length["5-LTR"]
            else:
                coord_start = int(query_hit["contig_stop"]) + int(query_hit["TE_stop"]) - consensus_length["5-LTR"]
            for other_hit in consensus_hits:
                if coord_start-500 <= int(other_hit["contig_stop"]) <= coord_start \
                and query_hit["contig"] == other_hit["contig"]:
                    return True
        return False

def BuildTE(query_hit, consensus_length, consensus_hits):
    TE_hits = []
    if SearchSense(query_hit["strand"], int(query_hit["TE_start"]), int(query_hit["TE_stop"]), consensus_length["5-LTR"]):           # change due to error of pairing with itself
        if query_hit["strand"] == "+" :
            coord_start = int(query_hit["contig_start"]) - int(query_hit["TE_start"])
        else:
            coord_start = int(query_hit["contig_start"]) + int(query_hit["TE_start"]) - consensus_length["complete"]
        for other_hit in consensus_hits:
            # print(other_hit["contig_start"])
            if coord_start <= int(other_hit["contig_start"]) <= coord_start+1.2*consensus_length["complete"] \
            and query_hit["contig"] == other_hit["contig"]:
                TE_hits.append(other_hit)
    else:
        if query_hit["strand"] == "+" :
            coord_start = int(query_hit["contig_stop"]) - int(query_hit["TE_stop"]) + consensus_length["complete"]
        else:
            coord_start = int(query_hit["contig_stop"]) + int(query_hit["TE_stop"])
        for other_hit in consensus_hits:
            if coord_start-1.2*consensus_length["complete"] <= int(other_hit["contig_stop"]) <= coord_start \
            and query_hit["contig"] == other_hit["contig"]:
                TE_hits.append(other_hit)
    return TE_hits



os.chdir(sequences_dir)
headers = ["index", "contig", "start", "stop", "length", "strand", "id_percentage", "TE_start", "TE_stop", "classification", "index", "start", "stop", "length", "strand", "id_percentage", "TE_start", "TE_stop"]
dico_count = {}

for consensus in dico_repeat_hits :
    print(consensus)
    os.chdir(consensus)
    output_consensus = open(consensus + ".LTR_classif.tsv", "w")
    output_consensus.write("\t".join(headers) + "\n")
    count = [0,0,0,0,0,0,0]
    hits_used = []
    list_lines = []
    
    for hit in dico_repeat_hits[consensus]:
        if hit in hits_used:
            continue
        else:
            TE_hits, status, count = ClassLTR(hit, dico_length[consensus], dico_repeat_hits[consensus], count)
            ref_line = [str(hit["index"]), hit["contig"], hit["contig_start"], hit["contig_stop"], str(int(hit["contig_stop"])-int(hit["contig_start"])), hit["strand"], str(hit["perc_id"]), hit["TE_start"], hit["TE_stop"], status]
            list_lines.append(ref_line)
            if len(TE_hits) == 0 :
                list_lines.append(["\tno_match\n"])
            else:
                new_line = ''
                for other_hit in TE_hits:
                    hits_used.append(other_hit)
                    for line in list_lines:
                        if str(other_hit["index"]) == line[0] and line != ref_line:
                            del list_lines[list_lines.index(line):list_lines.index(line)+2]
                            count = FixCount(line[-1], count)
                    line = [new_line, str(other_hit["index"]), other_hit["contig_start"], other_hit["contig_stop"], str(int(other_hit["contig_stop"])-int(other_hit["contig_start"])), other_hit["strand"], str(other_hit["perc_id"]), other_hit["TE_start"], other_hit["TE_stop"], "\n"]
                    list_lines.append(line)
                    new_line = 9*'\t'

    for line in list_lines:
        output_consensus.write("\t".join(line))
    output_consensus.close()
    dico_count[consensus] = count
    os.chdir("..")

output_count = open("annotation_counts.tsv", "w")
headers = ["consensus", "length", "trunc", "solo", "single", "paired", "fragmented", "shrunked", "complete"]
output_count.write("\t".join(headers) + "\n")

for consensus in dico_count:
    output_count.write(consensus + "\t" + str(dico_length[consensus]["complete"]) + "\t" + "\t".join(str(i) for i in dico_count[consensus]) + "\n")   

In [7]:
output_count = open("annotation_counts.tsv", "w")
headers = ["consensus", "length", "trunc", "solo", "single", "paired", "fragmented", "shrunked", "complete"]
output_count.write("\t".join(headers) + "\n")

for consensus in dico_count:
    output_count.write(consensus + "\t" + str(dico_length[consensus]["complete"]) + "\t" + "\t".join(str(i) for i in dico_count[consensus]) + "\n")   

### Step 4 : Extraction of LTR coordinates and sequences
After the classification, the goal of this program is to identify the age of insertion of each TE for each consensus. We need the TE sequences for it, and so TE coordinates.

In [ ]:
# structure of dico_LTRs_by_consensus :
# { consensus1 : [LTR_list], consensus2 : [LTR_list], ...}
# LTR_list = [{LTR1_hit1}, {LTR1_hit2}, {LTR15_hit1}, {LTR15_hit2}, ...]
# LTR1_hit1 = {id : index_status_5LTR, contig, contig_start, contig_stop, strand, id_percentage, TE_start, TE_stop}
# then we will add the sequence to that dictionnary

def Check5LTR(TE_start, TE_stop, strand, consensus_length):
    '''Check if the hit can be considered as a 5'LTR from its coordinates and its coverage of LTR'''
    if strand == "+":
        return TE_start < consensus_length["5-LTR"] \
           and min(TE_stop,consensus_length["5-LTR"]) - TE_start > 0.8*consensus_length["5-LTR"]
    else:
        return TE_stop < consensus_length["5-LTR"]\
           and min(TE_start,consensus_length["5-LTR"]) - TE_stop > 0.8*consensus_length["5-LTR"]

def Check3LTR(TE_start, TE_stop, strand, consensus_length):
    '''Check if the hit can be considered as a 3'LTR from its coordinates and its coverage of LTR'''
    if strand == "+":
        return TE_stop > consensus_length["complete"] - consensus_length["3-LTR"] \
           and TE_stop - max(TE_start,consensus_length["complete"]-consensus_length["3-LTR"]) > 0.5*consensus_length["3-LTR"]
    else:
        return TE_start > consensus_length["complete"] - consensus_length["3-LTR"] \
           and TE_start - max(TE_stop,consensus_length["complete"]-consensus_length["3-LTR"]) > 0.5*consensus_length["3-LTR"]

def CountRepetitions(list_hits, id):
    '''Some hits are considered as the same LTR for the same TE (possible overlapping between several TEs,
    considered as one TE). This function manage those repetitions by giving a new index with a number
    (ex : 15436_complete_5LTR.2)'''
    i=1
    new_id = id
    for hit in list_hits:
        if id in hit["id"]:
            i+=1
            new_id = id + "." + str(i)
    return new_id

def AddLTR(elements, id, contig):
    '''Add the LTR informations in dico_LTRs_by_consensus'''
    return {"id":id, "contig":contig,
            "contig_start":elements[11],
            "contig_stop":elements[12],
            "strand":elements[14],
            "id_percentage":elements[15],
            "TE_start":elements[16],
            "TE_stop":elements[17] }

def WriteLTR(getfasta_library, dico_LTR, consensus_length, contig_length):
    '''Write the coordinates file used by bedtools getfasta to generate fasta files.
    Coordinates are taking the entire predicted LTR and not the entire hit.
    coordinate_start = estimated minimal LTR coordinate (5' start or 3' stop) or first contig base
    coordinate_stop = estimated maximal LTR coordinate (5' stop or 3' start) or last contig base'''
    print(id)
    if dico_LTR["strand"] == "+":
        if "5LTR" in dico_LTR["id"]:
            coordinate_start = max(int(dico_LTR["contig_start"]) - int(dico_LTR["TE_start"]) - 50, 1)
            coordinate_stop = min(int(dico_LTR["contig_stop"]), coordinate_start+consensus_length["5-LTR"]+100)
        else:
            coordinate_stop = min(int(dico_LTR["contig_stop"]) + consensus_length["complete"] - int(dico_LTR["TE_stop"]) + 50, contig_length)
            coordinate_start = max(int(dico_LTR["contig_start"]), coordinate_stop-consensus_length["3-LTR"]-50)
    else:
        dico_LTR["strand"] = "-"
        if "5LTR" in dico_LTR["id"]:
            coordinate_stop = min(int(dico_LTR["contig_stop"]) + int(dico_LTR["TE_stop"]) + 50, contig_length)
            coordinate_start = max(int(dico_LTR["contig_start"]), coordinate_stop-consensus_length["5-LTR"]-50)
        else:
            coordinate_start = max(int(dico_LTR["contig_start"]) - consensus_length["complete"] + int(dico_LTR["TE_start"]) - 50, 1)
            coordinate_stop = min(int(dico_LTR["contig_stop"]), coordinate_start+consensus_length["3-LTR"]+50)
    line = [dico_LTR["contig"], str(coordinate_start), str(coordinate_stop), dico_LTR["id"], "0", dico_LTR["strand"]]
    getfasta_library.write("\t".join(line) + "\n")
    # remember the columns in bed files :
    # contig    contig_start    contig_stop     name    (score)   strand

# Taking fasta sequences for all contigs of the genome for their length (to avoid out of bounds TEs)
contigs = {seq_record.id: seq_record.seq for seq_record in SeqIO.parse(genome, "fasta")}
for contig in contigs:
    dico_length[contig] = len(contigs[contig])

os.chdir(sequences_dir)
dico_LTRs_by_consensus = {}
for consensus in os.listdir("."):
    getfasta_library = open(consensus + "/" + consensus + ".LTR_coordinates.bed", "w")
    print(consensus, dico_length[consensus])
    dico_LTRs_by_consensus[consensus] = []
    for line in open(consensus + "/" + consensus + ".LTR_classif.tsv", "r"):
        elements = line.split("\t")
        # skip trunc TEs and header
        if elements[10] == "no_match\n" or elements[9] == "classification":
            continue
        # elif elements[0] == "":
        #     dico_one_LTR = {}
        #     if Check5LTR and dico_one_LTR["5-LTR"] != {}:
        #         doc_one_LTR["5-LTR"] = {"contig", "contig-start", "contig-stop", "strand", "TE-start", "TE-stop"}
        #     elif Check3LTR and dico_one_LTR["3-LTR"] != {}:
        #         doc_one_LTR["3-LTR"] = {"contig", "contig-start", "contig-stop", "strand", "TE-start", "TE-stop"}
        else:
            # informations for a new TE
            if elements[0] != "":
                index = elements[0]         # index = line of hit in RepeatMasker file
                contig = elements[1]
                status = elements[9]        # status = LTR classification
            if Check5LTR(int(elements[16]), int(elements[17]), elements[14], dico_length[consensus]):
                # id will be used to identify LTRs in trees
                id = CountRepetitions(dico_LTRs_by_consensus[consensus], "%s_%s_5LTR" % (index, status))
                dico_LTRs_by_consensus[consensus].append(AddLTR(elements, id, contig))
                WriteLTR(getfasta_library, dico_LTRs_by_consensus[consensus][-1], dico_length[consensus], dico_length[contig])
            if Check3LTR(int(elements[16]), int(elements[17]), elements[14], dico_length[consensus]):       # no elif because complete TEs have both LTR
                id = CountRepetitions(dico_LTRs_by_consensus[consensus], "%s_%s_3LTR" % (index, status))
                dico_LTRs_by_consensus[consensus].append(AddLTR(elements, id, contig))
                WriteLTR(getfasta_library, dico_LTRs_by_consensus[consensus][-1], dico_length[consensus], dico_length[contig])
    getfasta_library.close()            

### Step 5 : Phylogeny of TEs
This step is done after a run of Extract_LTR_for_phylogeny.sh, which run bedtools getfasta and mafft.
Those runs need also to convert the alignments in .nex files (here done with Mesquite).

In [2]:
os.chdir(sequences_dir)

def RunMrBayes(consensus, count):
    '''Write the nexus file used to run MrBayes, and then proceed to run MrBayes in parallel
    version with 10 processors (need to be installed to shorten runs)'''
    os.chdir(consensus)
    nexus_file = "mrbayes_commands.nex"
    mrbayes_script = open(nexus_file, "w")
    file = "\texecute " + consensus + ".LTR_alignment.nex;"
    # The number of generations is generated proportionally to the number of LTRs sequences
    generations = "\tmcmc ngen=%s samplefreq=100 printfreq=100 diagnfreq=1000 nchains=5;" % str(count*1000)
    list_lines = ["#NEXUS\n",
                  "begin mrbayes;",
                  file,
                  "\tprset brlenspr=clock:uniform;",
                  "\tlset nst=2 rates=invgamma;",
                  generations,
                  "\tsump;",
                  "\tsumt;",
                  "end;"]
    mrbayes_script.write("\n".join(list_lines))
    mrbayes_script.close()
    os.system("mpirun -np 10 %s %s > mrbayes_run.log" % (mb, nexus_file))
    os.chdir("..")

# Get LTRs count for the number of MrBayes generations
count_file = open("../annotation_counts.tsv", "r")
list_counts = []
for line in count_file:
    if "paired" not in line:
        elements = line.split("\t")
        list_counts.append([elements[0], int(elements[-1][:-1])])

list_counts.sort(key=lambda x: (x[1]))      # shorter analysis will be done first
i=0
for consensus in list_counts:
    i+=1
    print(i, consensus[0])
    if i>62:                # this line will be used if you want to stop your analysis
        RunMrBayes(consensus[0], consensus[1])




1 c64_s1
2 c399_s1
3 c111_s2
4 c82_s1
5 c79_s1
6 c60_s1
7 c339_s1
8 c64_s2
9 c15_s1
10 c339_s2
11 c34_s2
12 c411
13 c124_s2
14 c384_s2
15 c117_s1
16 c79_s2
17 c384_s1
18 c46_s1
19 c419
20 c53_s2
21 c399_s2
22 c46_s2
23 c366
24 c320
25 c43_s1
26 c60_s2
27 c83_s2
28 c117_s2
29 c221_s2
30 c43_s2
31 c130_s2
32 c83_s1
33 c8_s2
34 c133
35 c194
36 c332
37 c15_s2
38 c71_s2
39 c111_s1
40 c347_s2
41 c53_s1
42 c62_s1
43 c71_s1
44 c204_s1
45 c57_s2
46 c244
47 c347_s1
48 c144_s2
49 c242_s1
50 c62_s2
51 c124_s1
52 c144_s1
53 c82_s2
54 c304
55 c361
56 c103_s2
57 c336_s1
58 c215_s1
59 c145
60 c226
61 c165_s1
62 c21_s2
63 c354_s1


hwloc/linux: Ignoring PCI device with non-16bit domain.
Pass --enable-32bits-pci-domain to configure to support such devices
(warning: it would break the library ABI, don't enable unless really needed).


64 c113


hwloc/linux: Ignoring PCI device with non-16bit domain.
Pass --enable-32bits-pci-domain to configure to support such devices
(warning: it would break the library ABI, don't enable unless really needed).


65 c57_s1


...

hwloc/linux: Ignoring PCI device with non-16bit domain.
Pass --enable-32bits-pci-domain to configure to support such devices
(warning: it would break the library ABI, don't enable unless really needed).


70 c55_s2


hwloc/linux: Ignoring PCI device with non-16bit domain.
Pass --enable-32bits-pci-domain to configure to support such devices
(warning: it would break the library ABI, don't enable unless really needed).


71 c67_s1


hwloc/linux: Ignoring PCI device with non-16bit domain.
Pass --enable-32bits-pci-domain to configure to support such devices
(warning: it would break the library ABI, don't enable unless really needed).


Step X : Context of insertion

- gene context (most important)
- recombination context
- TF context
- GC content
- non B conformation
- maybe methylation and polymorphism with flowcell